# Cleaning historical ERCOT fuel mix data

[Brian C. Keegan, Ph.D.](http://brianckeegan.com/)  
[Assistant Professor, Department of Information Science](https://www.colorado.edu/cmci/people/information-science/brian-c-keegan)  
University of Colorado Boulder  

Copyright and distributed under an [MIT License](https://opensource.org/licenses/MIT).

In [1]:
import numpy as np
import pandas as pd

import os, re

# %matplotlib inline
# import matplotlib.pyplot as plt
# import seaborn as sb

## Texas fuel generation data

Use the ERCOT 20201 "Fuel Mix Report" available from its "[Generation](http://www.ercot.com/gridinfo/generation)" page. Download the historical 2007-2020 data and unzip. Download the in-progress 2021 data. Put the 2021 file in the same unzipped directory as the 2007-2020 data.

Read in previous years' data, reshape, and combine to aid in comparisons. 

The path to the folder where the data you've downloaded is very likely different than my path.

In [2]:
path = '/Users/briankeegan/Downloads/FuelMixReport_PreviousYears/'

sorted(os.listdir(path))

['IntGenByFuel2007.xls',
 'IntGenByFuel2008.xls',
 'IntGenByFuel2009.xls',
 'IntGenByFuel2010.xls',
 'IntGenByFuel2011.xls',
 'IntGenByFuel2012.xls',
 'IntGenByFuel2013.xls',
 'IntGenByFuel2014.xls',
 'IntGenByFuel2015.xls',
 'IntGenByFuel2016.xlsx',
 'IntGenbyFuel2017.xlsx',
 'IntGenbyFuel2018.xlsx',
 'IntGenbyFuel2019.xlsx',
 'IntGenbyFuel2020.xlsx',
 'IntGenbyFuel2021.xlsx']

Define some `time_values` in 15-minute increments and put 00:00 at the end

In [3]:
time_vals = [str(t.time()) for t in pd.date_range(start='00:00',end='23:55',freq='15min')]
time_vals = time_vals[1:] + [time_vals[0]]

Clean up each file.

In [4]:
feb_hist_data = {}

for f in os.listdir(path):
    # Extract year from filename
    _year = re.findall('\w+(20\d+).xls',f)[0]
    
    # Open Excel file
    _xl = pd.ExcelFile(path + f)
    
    # Search for sheetname containing "Feb"
    _sheet_name = [s for s in _xl.sheet_names if "Feb" in s]
    
    # Open the Feb sheet
    _df = _xl.parse(_sheet_name[0])

    # Cast columns to strings because some are time objects (?!)
    _df.columns = _df.columns.astype(str)
    
    # Drop "Unnamed" columns
    _df = _df.drop(columns=[c for c in _df.columns if 'Unnamed:' in c])
    
    # Find the first column with a timestamp
    _min_col_pos = min([i for i,c in enumerate(_df.columns) if '0:15' in c])
    
    # Relabel columns
    _df.columns = _df.columns[:_min_col_pos].tolist() + time_vals

    # Some columns have Date-Fuel
    if type(_df.iloc[0,0]) == str:
        if ' - ' in _df.iloc[0,0]:
            _df[['Date','Fuel']] = _df.iloc[:,0].str.split(' - ',expand=True)
        elif '-' in _df.iloc[0,0]:
            _df[['Date','Fuel']] = _df.iloc[:,0].str.split('-',expand=True)
        elif '_' in _df.iloc[0,0]:
            _df[['Date','Fuel']] = _df.iloc[:,0].str.split('_',expand=True)
            
    # Cast Dates to strings
    _df['Date'] = pd.to_datetime(_df['Date']).astype('str')
    
    # Assign cleaned df to year
    feb_hist_data[_year] = _df

Rename and clean up values in various columns.

In [5]:
# Concatenate the DFs together into a big DF
hist_fuel_mix_df = pd.concat(feb_hist_data.values())

# Define a mapping for renaming Fuel values to consistent names
fuel_replace_d = {'Fuel':{
    'Gas_CC':'Gas-CC',
    'Wnd':'Wind',
    'Oth':'Other',
    'Sun':'Solar',
    'Gas_GT':'Gas'}}

# Replace the inconsistent values
hist_fuel_mix_df = hist_fuel_mix_df.replace(fuel_replace_d)

# Count the values in the Fuel column
hist_fuel_mix_df['Fuel'].value_counts()

Nuclear    424
Other      424
Hydro      424
Coal       424
Wind       424
Gas        424
Gas-CC     367
Solar      283
Biomass    283
Name: Fuel, dtype: int64

Reshape the data and save it.

In [6]:
# Melt the data from wide to long
hist_fuel_mix_melted_df = pd.melt(hist_fuel_mix_df,
                             id_vars=['Date','Fuel'],
                             value_vars=time_vals,
                             var_name='Time',
                             value_name='Generation'
                            )

print(hist_fuel_mix_melted_df.shape)
print(hist_fuel_mix_melted_df['Date'].min(),hist_fuel_mix_melted_df['Date'].max())

# Combine Date and Time columns
hist_fuel_mix_melted_df['Datetime'] = pd.to_datetime(hist_fuel_mix_melted_df['Date'].astype(str) + ' ' + hist_fuel_mix_melted_df['Time'].astype(str))

# Subset to non-redundant columns
hist_fuel_mix_melted_df = hist_fuel_mix_melted_df[['Datetime','Fuel','Generation']]

# Sort data
hist_fuel_mix_melted_df = hist_fuel_mix_melted_df.sort_values('Datetime').reset_index(drop=True)

# Save data
hist_fuel_mix_melted_df.to_csv('ercot_feb_fuel_mix.csv')

# Inspect
hist_fuel_mix_melted_df.head()

(333792, 4)
2007-02-01 2021-02-28


,Datetime,Fuel,Generation
0,2007-02-01,Wind,110.192505
1,2007-02-01,Coal,3568.420078
2,2007-02-01,Gas,3024.754058
3,2007-02-01,Hydro,3.677265
4,2007-02-01,Nuclear,1249.945931
